In [ ]:
import add_packages
import yaml, requests, re
from crewai import Agent, Task, Crew, Process
from pprint import pprint
from my_crewai import tools as my_tools
from my_langchain import chat_models

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
  configs_crewai = yaml.safe_load(f)

configs = configs_crewai
configs_agents_traveling = configs_crewai["agents"]["traveling"]
configs_tasks_traveling = configs_crewai["tasks"]["traveling"]

params = {
  "origin": "Atlanta, GA",
  "travel_dates": "November 2024",
  "destination": "Thialand",
  "interests": "Hiking, Yoga, Sightseeing",
}

# Base

In [ ]:
import re

with open(f"{add_packages.APP_PATH}/my_configs/crew_ai.yaml") as f:
  configs_crewai = yaml.safe_load(f)

configs = configs_crewai
configs_agents_traveling = configs_crewai["agents"]["traveling"]
configs_tasks_traveling = configs_crewai["tasks"]["traveling"]

params = {
    "origin": "Atlanta, GA",
    "travel_dates": "November 2024",
    "destination": "Thialand",
    "interests": "Hiking, Yoga, Sightseeing",
}

def substitute_placeholders_in_configs(configs, params):
    pattern = r'%(\w+)%'

    def repl(match):
        placeholder = match.group(1)
        if placeholder.lower() in params:
            return str(params[placeholder.lower()])
        return match.group(0)

    return {
        key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
        for key, value in configs.items()
    }


# Substitute placeholders in agent and task configurations
configs_agents_traveling_substituted = substitute_placeholders_in_configs(
    configs_agents_traveling, params)
configs_tasks_traveling_substituted = substitute_placeholders_in_configs(
    configs_tasks_traveling, params)

# Create agents
manager = Agent(
    **configs_agents_traveling_substituted["manager"],
    verbose=True,
    allow_delegation=True,
    llm=chat_models.chat_openai,
)

travel_agent = Agent(
    **configs_agents_traveling_substituted["travel_consultant"],
    tools=[
        my_tools.ToolsCalculator().calculate
    ],
    max_iter=7,
    llm=chat_models.chat_openai,
)

city_selection_expert = Agent(
    **configs_agents_traveling_substituted["city_selection_expert"],
    verbose=True,
    tools=[
        my_tools.ToolsBrowser().search_duckduckgo,
        my_tools.ToolsContent().read_content
    ],
    llm=chat_models.chat_openai,
)

local_tour_guide = Agent(
    **configs_agents_traveling_substituted["local_tour_guide"],
    verbose=True,
    tools=[
        my_tools.ToolsBrowser().search_duckduckgo,
        my_tools.ToolsContent().read_content
    ],
    llm=chat_models.chat_openai,
)

quality_control_expert = Agent(
    **configs_agents_traveling_substituted["quality_control_expert"],
    verbose=True,
    tools=[] + my_tools.tools_human,
    llm=chat_models.chat_openai,
)

# Create tasks
manager_task = Task(
    **configs_tasks_traveling_substituted["manage"],
    agent=manager,
    async_execution=False,
)

identify_city = Task(
    **configs_tasks_traveling_substituted["identify_city"],
    agent=city_selection_expert,
    async_execution=False
)

gather_city_info = Task(
    **configs_tasks_traveling_substituted["gather_city_info"],
    agent=city_selection_expert,
    async_execution=False
)

plan_itinerary = Task(
    **configs_tasks_traveling_substituted["plan_itinerary"],
    agent=travel_agent,
    async_execution=False,
)

quality_control = Task(
    **configs_tasks_traveling_substituted["quality_control"],
    agent=quality_control_expert,
    context=[plan_itinerary],
    async_execution=False
)

# Forming the crew with a hierarchical process including the manager
crew = Crew(
    agents=[
        manager,
        travel_agent,
        city_selection_expert,
        local_tour_guide,
        quality_control_expert],
    tasks=[manager_task,
           plan_itinerary,
           identify_city,
           gather_city_info,
           quality_control],
    process=Process.hierarchical,
    manager_llm=chat_models.chat_openai,
    verbose=2,
)

In [ ]:

# Kick off the crew's work
results = crew.kickoff()

# Print the results
print("Crew Work Results:")
print(results)

# final

In [ ]:
import re
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import requests

load_dotenv()


class CustomCrew:
  def __init__(self, params, configs, task_name) -> None:
    self.params = params
    self.configs_agents = configs["agents"][task_name]
    self.configs_tasks = configs["tasks"][task_name]
    self.agents = {}
    self.tasks = self.create_tasks()
    self.load_agents()
    self.crew = self.create_crew()

  def substitute_placeholders_in_configs(self, configs):
    pattern = r'%(\w+)%'

    def repl(match):
      placeholder = match.group(1)
      if placeholder.lower() in self.params:
        return str(self.params[placeholder.lower()])
      return match.group(0)
    return {
        key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
        for key, value in configs.items()
    }

  def load_agents(self):
    configs_agents_substituted = self.substitute_placeholders_in_configs(
        self.configs_agents)
    for agent_key, agent_config in configs_agents_substituted.items():
      agent_tools = self.get_agent_tools(agent_key)
      self.agents[agent_key] = Agent(
				**agent_config,
				tools=agent_tools,
				verbose=True,
				llm=chat_models.chat_openai  # Assuming chat_models is defined elsewhere
      )

  def get_agent_tools(self, agent_key):
    # Logic to assign tools based on agent key
    if agent_key == "manager":
      return [
          my_tools.ToolsCalculator().calculate
      ]
    elif agent_key == "travel_consultant":
      return [
          my_tools.ToolsCalculator().calculate,
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "city_selection_expert":
      return [
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "local_tour_guide":
      return [
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "quality_control_expert":
      return my_tools.tools_human
    else:
      return []

  def create_tasks(self):
    configs_tasks_substituted = self.substitute_placeholders_in_configs(
        self.configs_tasks)
    tasks = {}
    for task_key, task_config in configs_tasks_substituted.items():
      tasks[task_key] = Task(**task_config)
    return tasks

  def create_crew(self):
    return Crew(
        agents=list(self.agents.values()),
        tasks=list(self.tasks.values()),
        process=Process.hierarchical,
        manager_llm=chat_models.chat_openai,
        verbose=2
    )


# Assuming configs, params, and chat_models are defined
task_name = "traveling"
custom_crew = CustomCrew(params, configs, task_name)

In [ ]:
custom_crew.crew.kickoff()

In [ ]:
dict(custom_crew.agents["quality_control_expert"])

# test

In [ ]:
class CustomCrew:
  def __init__(self, params, configs, task_name) -> None:
    self.params = params
    self.configs_agents = configs["agents"][task_name]
    self.configs_tasks = configs["tasks"][task_name]
    self.agents = {}
    self.tasks = self.create_tasks()
    self.load_agents()
    self.crew = self.create_crew()

  def substitute_placeholders_in_configs(self, configs):
    pattern = r'%(\w+)%'

    def repl(match):
      placeholder = match.group(1)
      if placeholder.lower() in self.params:
        return str(self.params[placeholder.lower()])
      return match.group(0)
    return {
        key: {k: re.sub(pattern, repl, v) for k, v in value.items()}
        for key, value in configs.items()
    }

  def load_agents(self):
    configs_agents_substituted = self.substitute_placeholders_in_configs(
        self.configs_agents)

    for agent_key, agent_config in configs_agents_substituted.items():
      agent_tools = self.get_agent_tools(agent_key)

      allow_delegation = agent_config.get(
          "allow_delegation", "false").lower() == "true"
      agent_config.pop("allow_delegation", None)

      self.agents[agent_key] = Agent(
          **agent_config,
          tools=agent_tools,
          verbose=True,
          llm=chat_models.chat_openai,  # Assuming chat_models is defined elsewhere
          allow_delegation=allow_delegation
        )

  def get_agent_tools(self, agent_key):
    # Logic to assign tools based on agent key
    if agent_key == "manager":
      return [
          my_tools.ToolsCalculator().calculate
      ]
    elif agent_key == "travel_consultant":
      return [
          my_tools.ToolsCalculator().calculate,
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "city_selection_expert":
      return [
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "local_tour_guide":
      return [
          my_tools.ToolsBrowser().search_duckduckgo,
          my_tools.ToolsContent().read_content
      ]
    elif agent_key == "quality_control_expert":
      return my_tools.tools_human
    else:
      return []

  def create_tasks(self):
    configs_tasks_substituted = self.substitute_placeholders_in_configs(
        self.configs_tasks)
    tasks = {}
    
    for task_key, task_config in configs_tasks_substituted.items():
      async_execution = task_config.get(
        "async_execution", "false").lower() == "true"
      task_config.pop("async_execution", None)
      
      tasks[task_key] = Task(
          **task_config,
          async_execution=async_execution
      )
    return tasks

  def create_crew(self):
    return Crew(
        agents=list(self.agents.values()),
        tasks=list(self.tasks.values()),
        process=Process.hierarchical,
        manager_llm=chat_models.chat_openai,
        verbose=2
    )


# Assuming configs, params, and chat_models are defined
task_name = "traveling"
custom_crew = CustomCrew(params, configs, task_name)

In [ ]:
dict(custom_crew.tasks["identify_city"])